In [17]:
from groq import Groq
import re
import httpx

In [ ]:
groq_client = Groq(api_key="YOUR_GROQ_API_KEY")

In [20]:
response = groq_client.chat.completions.create(
            messages=[{
                "role": "user",
                "content": "Hello world"
            }],
            model="llama3-8b-8192",
            temperature=0.0
        )

In [21]:
response.choices[0].message.content

"Hello world! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [27]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = groq_client.chat.completions.create(
                        model="llama3-8b-8192",
                        temperature=0.0,
                        messages=self.messages
                    )
        return completion.choices[0].message.content

## ReAct Agent

In [28]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [29]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [30]:
abot = Agent(prompt)

In [31]:
result = abot("How much does a Toy Poodle weigh?")
print(result)

Thought: I need to look up the average weight of a Toy Poodle to answer this question.

Action: average_dog_weight: Toy Poodle

PAUSE


In [32]:
result = average_dog_weight('Toy Poodle')

In [33]:
result

'a toy poodles average weight is 7 lbs'

In [34]:
next_prompt = "Observation: {}".format(result)

In [35]:
abot(next_prompt)

'Answer: A Toy Poodle weighs 7 lbs.'

In [36]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

## Another example (Manually doing the Observation and feeding it to LLM)

In [37]:
abot = Agent(prompt)

In [38]:
question = """I have 2 dogs, a border collie and a scottish terrier. What is their combined weight"""
abot(question)

'Thought: I need to know the average weight of each breed to calculate their combined weight.\n\nAction: average_dog_weight: Border Collie\nPAUSE'

In [39]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))

In [40]:
next_prompt

'Observation: a Border Collies average weight is 37 lbs'

In [41]:
abot(next_prompt)

'Thought: Now I have the weight of my Border Collie, I need the weight of my Scottish Terrier.\n\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [42]:
next_prompt = "Observation: {}".format(average_dog_weight('Scottish Terrier'))

In [43]:
next_prompt

'Observation: Scottish Terriers average 20 lbs'

In [44]:
abot(next_prompt)

'Thought: Now I have the weights of both dogs, I can calculate their combined weight.\n\nAction: calculate: 37 + 20\nPAUSE'

In [45]:
next_prompt = "Observation: {}".format(eval("37+20"))
next_prompt

'Observation: 57'

In [46]:
abot(next_prompt)

'Thought: I have the combined weight of my two dogs, which is 57 pounds.\n\nAnswer: The combined weight of my two dogs, a Border Collie and a Scottish Terrier, is 57 pounds.'

Here is the complete message

In [47]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user',
  'content': 'I have 2 dogs,

## Automating the entire process

In [49]:
import re
action_re = re.compile(r'^Action: (\w+): (.*)$')


In [50]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print("--running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [52]:
question = """I have 2 dogs, a border collie and a scottish terrier. What is their combined weight"""
query(question=question)

Thought: I need to know the average weight of each breed to calculate their combined weight.

Action: average_dog_weight: Border Collie
PAUSE
--running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need to know the average weight of a Scottish Terrier.

Action: average_dog_weight: Scottish Terrier
PAUSE
--running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now I can calculate the combined weight of my two dogs.

Action: calculate: 37 + 20
PAUSE
--running calculate 37 + 20
Observation: 57
Answer: Your two dogs' combined weight is 57 pounds.
